In [1]:
import numpy as np
import os
import pandas as pd
import pathlib

# project_dir = pathlib.Path(__file__).parent.absolute()

# paths to data dirs
lc_train_path = "/home/chowjunwei37/Documents/data/training_set/noisy_train/"
params_train_path = "/home/chowjunwei37/Documents/data/training_set/params_train/"

In [2]:
file = "0100_01_01.txt"

In [3]:
df = pd.read_csv(lc_train_path + file, delimiter="\t", header=None, skiprows=6)
target = pd.read_csv(params_train_path + file, delimiter="\t", header=None, skiprows=2)

target = target.T
target.columns = ["label"]

total = pd.concat([df, target], axis=1)

corr = total.corr()["label"].abs().nlargest(101)
_ = corr.pop("label")

In [6]:
type(np.array(corr.index)[1])

int

In [22]:
index = np.array(corr.index)

with open("index.csv", "ab") as f:
    np.savetxt(f, index.reshape((1, 100)), delimiter=",")

In [5]:
import numpy as np 
import pandas as pd 

In [7]:
df = np.array(pd.read_csv("./outputs/indexes.csv", header=None))

In [9]:
df = df.flatten()

In [11]:
np.bincount(df)

array([38702, 38544, 38406, 38459, 38348, 38194, 38519, 38308, 38149,
       38455, 38387, 38475, 38371, 38504, 38381, 38261, 38463, 38685,
       38443, 38364, 38763, 38518, 38493, 38615, 38313, 38560, 38493,
       38450, 38328, 38273, 38124, 38385, 38315, 38415, 38396, 38358,
       38520, 38336, 38124, 38607, 38553, 38379, 38629, 38383, 38494,
       38545, 38423, 38284, 38291, 38295, 38583, 38213, 38410, 38560,
       38245, 38610, 38248, 38530, 38559, 38325, 38521, 38589, 38821,
       38732, 38408, 38817, 38646, 39133, 39086, 39665, 39272, 39940,
       39926, 40116, 40536, 40486, 40968, 41098, 41495, 41492, 41734,
       42127, 42059, 42593, 42745, 42976, 43203, 43191, 43730, 43407,
       43820, 44164, 44226, 44258, 44566, 44598, 44737, 45193, 45013,
       45537, 45750, 45810, 45535, 45783, 45813, 45904, 46097, 46031,
       46030, 45994, 46254, 46250, 46317, 46268, 46110, 45833, 46238,
       46086, 46452, 46442, 46700, 46778, 46644, 46537, 46865, 46709,
       46718, 46758,

In [12]:
df = pd.DataFrame(df)

In [19]:
with pd.option_context("display.max_rows", None):
    print(df[0].value_counts())

167    47122
162    47025
159    46987
164    46980
172    46942
166    46909
147    46901
154    46899
144    46875
165    46872
124    46865
171    46855
169    46846
128    46839
132    46825
131    46814
170    46812
137    46800
121    46778
163    46769
155    46759
127    46758
168    46756
141    46748
153    46740
139    46737
150    46735
160    46727
151    46721
126    46718
125    46709
138    46703
149    46703
158    46702
120    46700
130    46694
129    46690
148    46684
175    46675
152    46674
122    46644
161    46627
136    46626
133    46622
135    46608
173    46595
140    46573
134    46566
157    46539
123    46537
142    46482
143    46472
146    46456
145    46455
118    46452
156    46443
119    46442
176    46353
112    46317
180    46314
177    46304
113    46268
110    46254
111    46250
185    46243
116    46238
178    46209
179    46202
174    46195
190    46138
114    46110
182    46108
106    46097
186    46089
117    46086
181    46077
184    46056

In [28]:
temp = df[0].unique()
temp.sort_values()

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [32]:
len(temp)

300